In [ ]:
import nltk
import tensorflow as tf
from keras import backend as K
import keras
import string
from sklearn.preprocessing import LabelEncoder
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam
from sklearn.model_selection import KFold
nltk.download('brown')
nltk.download('universal_tagset')


In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

AUTO = tf.data.experimental.AUTOTUNE
print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
tag_sent = nltk.corpus.brown.tagged_sents(tagset='universal')
print(len(tag_sent))
# po=set([y[1] for x in tag_sent for y in x])
sentences=[[x[0] for x in y] for y in tag_sent]
#print(len(sentences))
tags=[[x[1] for x in y] for y in tag_sent]
#print(sentences[0])

# tmp1,tmp2=[],[]
# for x,x1 in zip(sentences,tags):
#     new_sent=[]
#     new_punc=[]
#     for y,y1 in zip(x,x1):
#     # if y in '!"#$%&()*+,-/:;<=>?@[\\]^_\'`{|}~\t\n':
#         if len(y.translate({ord(x):None for x in '!."#$%&()*+,-/:;<=>?@[\\]^_\'`{|}~\t\n'}))==0:
#             pass
#         else:
#             new_sent.append(y)
#             new_punc.append(y1)
#             tmp1.append(new_sent)
#             tmp2.append(new_punc)
# sentences=tmp1
# tags=tmp2

def concat(li):
    sent=''
    for x in li:
        if len(sent)!=0:
            sent=sent+' '
        sent=sent+x
    return sent
MAX_LEN=60
sentences=[concat(x).lower() for x in sentences]
#print(sentences[0])

In [ ]:
NUM_WORDS=50000
tok=keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,oov_token='<OOV>',filters='\t\n')
tok.fit_on_texts(sentences)
# print(tok.get_config())
# print(tok.word_index)
sentences=tok.texts_to_sequences(sentences)

sentences=keras.preprocessing.sequence.pad_sequences(sentences,padding='post',maxlen=MAX_LEN)
# print(a[4])

# print(len(set([x for y in tags for x in y]+['$'])))

#print('koko')
tags=keras.preprocessing.sequence.pad_sequences(tags,padding='post',value='$',maxlen=MAX_LEN,dtype=object).tolist()
#print(tags[0])
# NUM_TAGS=set([x for y in tags for x in y]+['$'])
#print(len(tags[0]))
tok_tag=keras.preprocessing.text.Tokenizer(oov_token='<UNK>',filters='\t\n')


tok_tag.fit_on_texts(tags)
num_tags=len(tok_tag.word_index)
tags=tok_tag.texts_to_sequences(tags)
tags=np.array(tags)
print('done')

In [ ]:
#print(tok_tag.word_index)

In [ ]:
#print(tok_tag.word_index['$'])
def ignore_accuracy_of_class(class_to_ignore=0):
    def acc(y_true, y_pred):
        y_true_class=tf.cast(y_true,tf.int64)
#         y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
#         print((y_true_class).dtype)
#         print((y_pred_class).dtype)
        ignore_mask = K.cast(K.not_equal(y_pred_class, class_to_ignore), 'int32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'int32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy

    return acc
custacc=ignore_accuracy_of_class(tok_tag.word_index['$'])

In [ ]:
def remove_eos(actual,pred):
    ansact,anspred=[],[]
    for x,y in zip(actual,pred):
        if x=='$':
            return (ansact,anspred)
        else :
            ansact.append(x)
            anspred.append(y)
    return (ansact,anspred)

def get_lists(actual,pred): #give the input as list of list
    global tok_tag
    rev = dict(map(reversed, tok_tag.word_index.items()))
    actual=[[rev[x] for x in y] for y in actual]
    pred=[[rev[x] for x in y] for y in pred]
    cleaned=[remove_eos(x,y) for x,y in zip(actual,pred)]
    a1,a2=[],[]
    for x in cleaned:
        for a in x[0]:
            a1.append(a)
        for a in x[1]:
            a2.append(a)
    return a1,a2

In [ ]:
kfold = KFold(5, True, 1)
y_trues,y_preds=[],[]
miswords,mistags,mispreds=[],[],[]
for train,test in kfold.split(sentences):
    with strategy.scope():
        model = Sequential()
        model.add(InputLayer(input_shape=(MAX_LEN,), dtype='int32'))
        # model.add(InputLayer(input_shape=(MAX_LEN,)))
        model.add(Embedding(NUM_WORDS+1, 64,trainable=True))
        model.add(Bidirectional(LSTM(64, return_sequences=True,kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-4, l2=1e-3))))
        model.add(TimeDistributed(Dense(num_tags+1,activation='softmax',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-4, l2=1e-3))))
    model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(0.001),metrics=[custacc])
#     print(len(train))
    xtrain=sentences[train]
#     print(np.shape(xtrain))
    ytrain=tags[train]
    xtest=sentences[test]
    ytest=tags[test]
    model.fit(x=xtrain,y=ytrain,validation_data=(xtest,ytest),epochs=37,batch_size = 128 * strategy.num_replicas_in_sync)
    ypred=model.predict(xtest)
#     print(xtest.shape)
#     print(ypred.shape)
    ypred=np.argmax(ypred,axis=2)
    for i in range(min(np.shape(xtest)[0],np.shape(ypred)[0])):
#         print(np.shape(ypred[i,:]))
#         print(np.shape(ypred))
        for j in range(np.shape(ypred[i,:])[0]):
            if ypred[i][j]!=ytest[i][j] and ytest[i][j]!=tok_tag.word_index['$']:
                miswords.append(xtest[i,max(0,j-1):j+2])
                mistags.append(ytest[i,max(0,j-1):j+2])
                mispreds.append(ypred[i,max(0,j-1):j+2])
    #misclassis.append(misclassi)
#     print(ypred.shape)
#     print(ytest.shape)
    ypred=ypred.tolist()
    ytest=ytest.tolist()
    a1,a2=get_lists(ytest,ypred)
    y_trues.append(a1)
    y_preds.append(a2)
    

In [ ]:
#print(len(misclassis[0]))
idxword={v:k for k,v in tok.word_index.items()}
idxtag={v:k for k,v in tok_tag.word_index.items()}
wordtri=[[idxword.get(z,'<UNK>') for z in x]for x in miswords]
tagtri=[[idxtag.get(z,'<UNK>') for z in x]for x in mistags]
predtri=[[idxtag.get(z,'<UNK>') for z in x]for x in mispreds]
# print(tagtri[30:40])
# print(predtri[30:40])
# print(wordtri[30:40])

In [ ]:
di={}
for x in tagtri:
    di[tuple(x)]=di.get(tuple(x),0)+1
inv=[(k,v) for v,k in di.items()]
inv.sort(reverse=True)
print(inv)

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt

def get_report(y_true, y_pred, classes):
    clf_report = classification_report(y_true, y_pred, labels=classes, zero_division=0)
    clf_report = clf_report.replace('\n\n', '\n')
    clf_report = clf_report.replace('macro avg', 'macro_avg')
    clf_report = clf_report.replace('micro avg', 'micro_avg')
    clf_report = clf_report.replace('weighted avg', 'weighted_avg')
    clf_report = clf_report.replace(' / ', '/')
    lines = clf_report.split('\n')

    class_names, plotMat, support = [], [], []
    for line in lines[1:]:
        t = line.strip().split()
        if len(t) < 2:
            continue
        v = [float(x) for x in t[1: len(t) - 1]]
        if len(v) == 1 : v = v * 3
        support.append(int(t[-1]))
        class_names.append(t[0])
        plotMat.append(v)
    plotMat = np.array(plotMat)
    support = np.array(support)
    return class_names, plotMat, support

def get_scores(y_true, y_pred, classes):
    correct, wrong = {}, {}
    for tag in classes:
        correct[tag] = 0
        wrong[tag] = 0
        
    for tag, pred in zip(y_true, y_pred):
        if tag in correct and tag == pred:
            correct[tag] += 1
        elif tag in wrong:
            wrong[tag] += 1
            
    scores = []
    total = len(y_true)
    for tag in classes:
        cur = np.array([correct[tag], wrong[tag]])
        scores.append(cur / total)
    return np.array(scores)
    
def plot_confusion_matrix(classes, mat, normalize=True, cmap=plt.cm.Blues):
    cm = np.copy(mat)
    title = 'Confusion Matrix (without normalization)'
    if normalize:
        cm = cm.astype('float') / np.sum(cm, axis=1, keepdims=True)
        title = title.replace('without', 'with')
    plt.clf()    
    fig, ax = plt.subplots(figsize=(20,10))
    ax.set_title(title, y=-0.06, fontsize=22)
    ax.xaxis.set_ticks_position('top')
    ax.xaxis.set_label_position('top')
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = np.max(cm) / 2
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            color = "white" if (cm[i, j] > thresh) else "black"
            plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color=color)
    plt.ylabel('True label',fontsize=22)
    plt.xlabel('Predicted label', fontsize=22)
    plt.tight_layout()
    plt.savefig('confusion_matrix.png', bbox_inches="tight", transparent=True)
    
def plot_clf_report(classes, plotMat, support, cmap=plt.cm.Blues):
    title = 'Classification Report'
    xticklabels = ['Precision', 'Recall', 'F1-score']
    yticklabels = ['{0} ({1})'.format(classes[idx], sup) for idx, sup in enumerate(support)]
    plt.clf()
    fig, ax = plt.subplots(figsize=(20,10))
    ax.set_title(title, y=-0.06, fontsize=22)
    ax.xaxis.set_ticks_position('top')
    ax.xaxis.set_label_position('top')
    ax.xaxis.set_tick_params(labelsize=18)
    ax.yaxis.set_tick_params(labelsize=14)
    plt.imshow(plotMat, interpolation='nearest', cmap=cmap, aspect='auto')
    plt.colorbar()
    plt.xticks(np.arange(3), xticklabels, rotation=0)
    plt.yticks(np.arange(len(classes)), yticklabels)

    thresh = np.max(plotMat) / 2
    for i in range(plotMat.shape[0]):
        for j in range(plotMat.shape[1]):
            color = "white" if (plotMat[i, j] > thresh) else "black"
            plt.text(j, i, format(plotMat[i, j], '.2f'), horizontalalignment="center", color=color, fontsize=14)

    plt.xlabel('Metrics',fontsize=22)
    plt.ylabel('Classes',fontsize=22)
    plt.tight_layout()
    plt.savefig('classification_report.png', bbox_inches="tight", transparent=True)
    
def plot_tag_scores(classes, scores, normalize=True):
    plt.clf()
    width = 0.45
    fig, ax = plt.subplots(figsize=(20,10))
    ax.xaxis.set_tick_params(labelsize=18, rotation=25)
    ax.yaxis.set_tick_params(labelsize=18)
    range_bar1 = np.arange(len(classes))
    rects1 = ax.bar(range_bar1, tuple(scores[:, 0]), width, color='b')
    rects2 = ax.bar(range_bar1 + width, tuple(scores[:, 1]), width, color='r')

    ax.set_ylabel('Scores',fontsize=22)
    ax.set_title('Tag scores', fontsize=22)
    ax.set_xticks(range_bar1 + width / 2)
    ax.set_xticklabels(classes)

    ax.legend((rects1[0], rects2[0]), ('Correct', 'Wrong'), fontsize=20)
    plt.legend()
    plt.savefig('tag_scores.png', bbox_inches="tight", transparent=True)
    plt.show()

In [ ]:
classes = labels
class_names = None
report = None
support = None
cm, scores = None, None
cnt = 0
print(classes)
for y_true, y_pred in zip(y_trues, y_preds):
    class_names, report_, support_ = get_report(y_true, y_pred, classes)
    cm_ = confusion_matrix(y_true, y_pred, labels=classes)
    scores_ = get_scores(y_true, y_pred, classes)
    
    if report is None : report = np.zeros_like(report_, dtype=np.float64)
    report += report_
    
    if support is None : support = np.zeros_like(support_, dtype=np.float64)
    support += support_
    
    if cm is None : cm = np.zeros_like(cm_, dtype=np.float64)
    cm += cm_
    
    if scores is None : scores = np.zeros_like(scores_, dtype=np.float64)
    scores += scores_
    
    cnt += 1
    
report /= cnt
support /= cnt
cm /= cnt
scores /= cnt

In [ ]:
plot_clf_report(class_names, report, support)

In [ ]:
plot_confusion_matrix(classes, cm)

In [ ]:
plot_tag_scores(classes, scores)